In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

VERSION = '99_v1_03_03_14_v1_12'
os.makedirs('result', exist_ok=True)

fold_list = [0,1,2,3,4]

for fold in fold_list:
    model_path_list = [
        #f'../../14_Baseline4/exp/result/14_v1_01/pseudo_fold{fold}.csv', # deberta-v3-base
        #f'../../14_Baseline4/exp/result/14_v1_08/pseudo_fold{fold}.csv', # deberta-base
        #f'../../14_Baseline4/exp/result/14_v1_09/pseudo_fold{fold}.csv', # roberta-base
        #f'../../14_Baseline4/exp/result/14_v1_10/pseudo_fold{fold}.csv', # deberta-v3-large
        #f'../../14_Baseline4/exp/result/14_v1_11/pseudo_fold{fold}.csv', # xlm-roberta-base
        f'../../14_Baseline4/exp/result/14_v1_12/pseudo_fold{fold}.csv', # deberta-large
        #f'../../14_Baseline4/exp/result/14_v1_13/pseudo_fold{fold}.csv', # roberta-large
        #f'../../14_Baseline4/exp/result/14_v1_14/pseudo_fold{fold}.csv', # xlm-roberta-large
        #f'../../14_Baseline4/exp/result/14_v1_15/pseudo_fold{fold}.csv', # deberta-v2-xlarge
        #f'../../14_Baseline4/exp/result/14_v1_16/pseudo_fold{fold}.csv', # deberta-xlarge
        #f'../../14_Baseline4/exp/result/14_v2_01/pseudo_fold{fold}.csv', # deberta-v3-base, seed200
    ]
    
    best_weights = {}
#     best_weights = {
#         'cohesion': np.array([0.16240573, 0.08282686, 0.01624057, 0.11976722, 0.01624057,
#                 0.16240573, 0.0784168 , 0.07015325, 0.12913753, 0.16240573]),
#          'syntax': np.array([0.15225017, 0.02024249, 0.04478987, 0.19634784, 0.02024249,
#                 0.2024249 , 0.05304506, 0.14970968, 0.14070501, 0.02024249]),
#          'vocabulary': np.array([0.24103641, 0.02721076, 0.02721076, 0.27210756, 0.08951406,
#                 0.08808241, 0.04703807, 0.07278205, 0.10780717, 0.02721076]),
#          'phraseology': np.array([0.20208263, 0.02020826, 0.02020826, 0.20208263, 0.02020826,
#                 0.17526739, 0.02020826, 0.09516657, 0.17637372, 0.06819401]),
#          'grammar': np.array([0.20891405, 0.03402119, 0.03402119, 0.34021194, 0.03402119,
#                 0.03402119, 0.08572859, 0.07334767, 0.12169177, 0.03402119]),
#          'conventions': np.array([0.17648661, 0.01781882, 0.01781882, 0.13360242, 0.09537523,
#                 0.12887179, 0.05749159, 0.11134062, 0.17818818, 0.08300594])
#     }

    df_list = [
        pd.read_csv(model_path) for model_path in model_path_list
    ]

    base_df = df_list[0]
    if 'text_id' not in base_df.columns:
        base_df['text_id'] = base_df['id'].values

    for df in df_list[1:]:
        try:
            df = base_df[['text_id']].merge(df, how='left', on='text_id')
        except:
            df['text_id'] = df['id'].values
            df = base_df[['text_id']].merge(df, how='left', on='text_id')


    num_models = len(model_path_list)

    TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

    preds_list = []
    for col in TARGET_COLS:
        _preds = 0
        for i,df in enumerate(df_list):
            if best_weights:
                _preds += df[col].values * best_weights[col][i]
            else:
                _preds += df[col].values
        preds_list.append(_preds)
    preds = np.stack(preds_list, axis=0).transpose(1,0)
    print(preds.shape)

    for i,col in enumerate(TARGET_COLS):
        base_df[col] = preds[:,i]
    
    fb1_df = pd.read_csv('../../00_EDA/00_v1_07/result/similar_df.csv').rename(columns={"id":"text_id"})
    base_df = base_df.merge(fb1_df, on="text_id", how="left")
    base_df.to_csv(f'result/{VERSION}_pseudo_fold{fold}.csv', index=False)

(15142, 6)
(15142, 6)
(15142, 6)
(15142, 6)
(15142, 6)


In [2]:
fold = 0
pd.read_csv(f'result/{VERSION}_pseudo_fold{fold}.csv').head(2)

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions,full_text
0,0000D23A521A,3.529826,3.832685,3.914495,4.077552,3.997399,3.730392,"Some people belive that the so called ""face"" o..."
1,00066EA9880D,4.041693,3.887161,4.247053,4.130254,3.756133,3.719355,Driverless cars are exaclty what you would exp...
